In [1]:
from toolbox import *
import warnings
import argparse
import random
import os
import pandas as pd
import numpy as np
import cv2
import librosa
import time
import re
from timeout_decorator import timeout
import json

import matplotlib.pyplot as plt
import numpy as np
from ConfigSpace import (
    Categorical,
    Configuration,
    ConfigurationSpace,
    EqualsCondition,
    Float,
    InCondition,
    Integer,
)

import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from sklearn.datasets import load_digits
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
import itertools
import xgboost as xgb

from sklearn.preprocessing import scale
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler
from skopt.callbacks import DeadlineStopper

from smac import MultiFidelityFacade as MFFacade
from smac import Scenario
from smac.facade import AbstractFacade
from smac.intensifier.hyperband import Hyperband
from smac.intensifier.successive_halving import SuccessiveHalving

import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import Dataset
import torch.optim as optim
import torchaudio
import torchaudio.transforms as trans
import re

from line_profiler import LineProfiler
from pathlib import Path


warnings.filterwarnings("ignore", category=RuntimeWarning)

# Load data

In [2]:
nodes_combination = [20, 100, 180, 260, 340, 400]
dataset_indices_max = 16
max_shape_to_run = 10000
alpha_range_nn = [0.001, 0.01, 0.1]
subsample = [0.5, 0.8, 1.0]

In [3]:
dataset_indices = list(range(dataset_indices_max))
dict_data_indices = {dataset_ind: {} for dataset_ind in dataset_indices}
encode_cnt = 0

X_data_list = []
y_data_list = []
dataset_names = []
def import_datasets():
    SUITE_ID = [337]
    for i in SUITE_ID:
        benchmark_suite = openml.study.get_suite(i)
        for task_id in benchmark_suite.tasks:  # iterate over all tasks
            task = openml.tasks.get_task(task_id)  # download the OpenML task
            dataset = task.get_dataset()
            X, y, categorical_indicator, attribute_names = dataset.get_data(
                dataset_format="dataframe", target=dataset.default_target_attribute
            )   

            # ### Covert labels to numerical values
            # le = LabelEncoder()
            # y_encoded = le.fit_transform(y)
            # y = pd.DataFrame(y_encoded)

            X_data_list.append(X)
            y_data_list.append(y)
            dataset_names.append(dataset.name)

            # print(" ")
            # print(" SUITE_ID:", i)
            # print("X_data_list length:", len(X_data_list))
            # print(" ")
    
import_datasets()

train_x_list = X_data_list.copy()
train_y_list = y_data_list.copy()
val_x_list = X_data_list.copy()
val_y_list = y_data_list.copy()

for dataset_index, dataset in enumerate(dataset_indices):
    print("\n\nCurrent Dataset: ", dataset)

    X = X_data_list[dataset]
    y = y_data_list[dataset]

    if X.shape[0] > max_shape_to_run:
        X, y = sample_large_datasets(X, y)
    
    np.random.seed(dataset_index)
    dict_data_indices = find_indices_train_val_test(
        X.shape[0], dict_data_indices=dict_data_indices, dataset_ind=dataset_index
    )
    train_indices = dict_data_indices[dataset_index]["train"]
    val_indices = dict_data_indices[dataset_index]["val"]

    ### Covert labels to numerical values
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    # y = pd.DataFrame(y_encoded)
    y = y_encoded

    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)
    # print(X.dtypes)

    ### Convert categories features to numerical features
    # print("X shape: ", X.shape)
    categorical_columns = X.select_dtypes(include=['object']).columns
    numeric_columns = X.select_dtypes(include=['number']).columns

    encoder = OneHotEncoder(sparse_output=False)
    if len(categorical_columns) > 0:
        X_encoded_strings = encoder.fit_transform(X[categorical_columns])

        X = np.hstack((X[numeric_columns].values, X_encoded_strings))
        print("Encoded", len(categorical_columns), " columns")
        encode_cnt += 1
    else:
        print("No string columns to encode")
    
    # print("X_encoded shape: ", X.shape)
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    # X = pd.DataFrame(X)
    train_x_list[dataset] = X[train_indices]
    train_y_list[dataset] = y[train_indices]
    val_x_list[dataset] = X[val_indices]
    val_y_list[dataset] = y[val_indices]
    print("X scalered")




Current Dataset:  0
No string columns to encode
X scalered


Current Dataset:  1
No string columns to encode
X scalered


Current Dataset:  2
No string columns to encode
X scalered


Current Dataset:  3
No string columns to encode
X scalered


Current Dataset:  4
No string columns to encode
X scalered


Current Dataset:  5
No string columns to encode
X scalered


Current Dataset:  6
No string columns to encode
X scalered


Current Dataset:  7
No string columns to encode
X scalered


Current Dataset:  8
No string columns to encode
X scalered


Current Dataset:  9
No string columns to encode
X scalered


Current Dataset:  10
No string columns to encode
X scalered


Current Dataset:  11
No string columns to encode
X scalered


Current Dataset:  12
No string columns to encode
X scalered


Current Dataset:  13
No string columns to encode
X scalered


Current Dataset:  14
No string columns to encode
X scalered


Current Dataset:  15
No string columns to encode
X scalered


In [4]:
train_x = train_x_list[0]
train_y = train_y_list[0]
val_x = val_x_list[0]
val_y = val_y_list[0]
num_class = len(np.unique(train_y))
# print(np.unique(train_y))
# print(type(train_x))
# print(type(train_y))
print(dataset_names)
print(len(dataset_names))
print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)

# categorical_columns = train_x.select_dtypes(include=['category']).columns

# if not categorical_columns.empty:
#     print("There are categorical columns:", categorical_columns)
# else:
#     print("No categorical columns found.")

# print(train_x.head(5))
# print(train_x['day'].cat.categories.is_numeric())

['credit', 'electricity', 'covertype', 'pol', 'house_16H', 'MagicTelescope', 'bank-marketing', 'MiniBooNE', 'Higgs', 'eye_movements', 'Diabetes130US', 'jannis', 'default-of-credit-card-clients', 'Bioresponse', 'california', 'heloc']
16
(5000, 10)
(5000,)
(2500, 10)
(2500,)


# Models

In [4]:
RF = RandomForestClassifier(n_estimators=100, random_state=317)
XGBT = xgb.XGBClassifier(n_estimators=100, random_state=317)
TabNet = TabNetClassifier(n_d=8, n_a=8, n_steps=5, gamma=1.3, n_independent=2, n_shared=2, seed=317, optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=1e-2), scheduler_params={"step_size":50, "gamma":0.9}, scheduler_fn=torch.optim.lr_scheduler.StepLR, mask_type='entmax', verbose=0)

In [ ]:
class XGBTWrapper(BaseEstimator):
    def __init__(self, n_estimators=100, max_depth=2, seed= 317, min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.5, learning_rate=0.1, objective='binary:logistic', colsample_bylevel=0.5, colsample_bynode=0.5):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.seed= seed
        self.min_child_weight = min_child_weight
        self.gamma = gamma
        self.subsample = subsample
        self.colsample_bytree = colsample_bytree
        self.colsample_bylevel = colsample_bylevel
        self.colsample_bynode = colsample_bynode
        self.learning_rate = learning_rate
        self.objective = objective
        # self.num_class = num_class
        self.model = xgb.XGBClassifier(n_estimators=self.n_estimators, max_depth=self.max_depth, seed=self.seed, min_child_weight=self.min_child_weight, gamma=self.gamma, subsample=self.subsample, colsample_bytree=self.colsample_bytree, colsample_bylevel=self.colsample_bylevel, colsample_bynode=self.colsample_bynode ,learning_rate=self.learning_rate, objective=self.objective)

    @property
    def configspace(self) -> ConfigurationSpace:
        cs = ConfigurationSpace()
        n_estimators = Integer("n_estimators", (100, 1200), default=100)
        max_depth = Integer("max_depth", (2, 21), default=2)
        min_child_weight = Integer("min_child_weight", (1, 10), default=1)
        gamma = Float("gamma", (0.1, 1.0), default=0.1)
        subsample = Float("subsample", (0.5, 1.0), default=0.8)
        colsample_bytree = Float("colsample_bytree", (0.3, 1.0), default=0.6)
        colsample_bylevel = Float("colsample_bylevel", (0.3, 1.0), default=0.6)
        colsample_bynode = Float("colsample_bynode", (0.3, 1.0), default=0.6)
        learning_rate = Float("learning_rate", (0.001, 0.3), default=0.1)
        cs.add_hyperparameters([n_estimators, max_depth, min_child_weight, gamma, subsample, colsample_bytree, colsample_bylevel, colsample_bynode, learning_rate])
        return cs
    
    def fit(self, config: Configuration, seed: int = 0, budget: int = 250) -> float: 
        config = dict(config)  
        self.model.set_params(**config)
        X = train_x
        y = train_y
        # print("X shape: ", X.shape)
        # print("y shape: ", y.shape)
        self.model.fit(X, y)
        preds = self.model.predict(X)
        scores = accuracy_score(y, preds)
        
        return 1 - scores
    

In [ ]:
@timeout(900)
def main():
    GBT = XGBTWrapper()

    facades: list[AbstractFacade] = []
    for intensifier_object in [Hyperband]:

        scenario = Scenario(
            GBT.configspace,
            walltime_limit=600,
            output_directory=Path("smac_hyperband_output_budget_10mins_XGBT"),
            n_trials=10000,
            min_budget=100,
            max_budget=1000,
            n_workers=8,

        )

        initial_design = MFFacade.get_initial_design(scenario, n_configs=5)
        intensifier = intensifier_object(scenario, incumbent_selection="highest_budget")

        smac = MFFacade(
            scenario,
            GBT.fit,
            initial_design=initial_design,
            intensifier=intensifier,
            overwrite=True,
        )

        print("optimiizing")
        print(type(smac), "|", smac)
        incumbent = smac.optimize()
        print("incumbent:", incumbent)
        default_cost = smac.validate(GBT.configspace.get_default_configuration())
        print(f"Default cost ({intensifier.__class__.__name__}): {default_cost}")
        incumbent_cost = smac.validate(incumbent)
        print(f"Incumbent cost ({intensifier.__class__.__name__}): {incumbent_cost}")

        facades.append(smac)
        for arrt in dir(smac):
            if not arrt.startswith("_"):
                print(arrt, getattr(smac, arrt))

    print("facades:", facades)



if __name__ == "__main__":
    # with open('smac_results_2h.txt', "w") as f:
    #     pass
    profiler = LineProfiler()
    profiler.add_function(main)
    profiler.enable()

    main()

    profiler.disable()
    profiler.print_stats()

In [40]:
params_dict_XGBT = {}
for i in range(len(train_x_list)):
    train_x = train_x_list[i]
    train_y = train_y_list[i]
    
    class XGBTWrapper(BaseEstimator):
        def __init__(self, n_estimators=100, max_depth=2, seed= 317, min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.5, learning_rate=0.1, objective='binary:logistic', colsample_bylevel=0.5, colsample_bynode=0.5):
            self.n_estimators = n_estimators
            self.max_depth = max_depth
            self.seed= seed
            self.min_child_weight = min_child_weight
            self.gamma = gamma
            self.subsample = subsample
            self.colsample_bytree = colsample_bytree
            self.colsample_bylevel = colsample_bylevel
            self.colsample_bynode = colsample_bynode
            self.learning_rate = learning_rate
            self.objective = objective
            # self.num_class = num_class
            self.model = xgb.XGBClassifier(n_estimators=self.n_estimators, max_depth=self.max_depth, seed=self.seed, min_child_weight=self.min_child_weight, gamma=self.gamma, subsample=self.subsample, colsample_bytree=self.colsample_bytree, colsample_bylevel=self.colsample_bylevel, colsample_bynode=self.colsample_bynode ,learning_rate=self.learning_rate, objective=self.objective)

        @property
        def configspace(self) -> ConfigurationSpace:
            cs = ConfigurationSpace()
            n_estimators = Integer("n_estimators", (100, 1500), default=100)
            max_depth = Integer("max_depth", (2, 21), default=2)
            min_child_weight = Integer("min_child_weight", (1, 10), default=1)
            gamma = Float("gamma", (0.1, 1.0), default=0.1)
            subsample = Float("subsample", (0.5, 1.0), default=0.8)
            colsample_bytree = Float("colsample_bytree", (0.3, 1.0), default=0.6)
            colsample_bylevel = Float("colsample_bylevel", (0.3, 1.0), default=0.6)
            colsample_bynode = Float("colsample_bynode", (0.3, 1.0), default=0.6)
            learning_rate = Float("learning_rate", (0.001, 0.3), default=0.1)
            cs.add_hyperparameters([n_estimators, max_depth, min_child_weight, gamma, subsample, colsample_bytree, colsample_bylevel, colsample_bynode, learning_rate])
            return cs
        
        def fit(self, config: Configuration, seed: int = 0, budget: int = 250) -> float: 
            config = dict(config)  
            self.model.set_params(**config)
            X = train_x
            y = train_y
            # print("X shape: ", X.shape)
            # print("y shape: ", y.shape)
            self.model.fit(X, y)
            preds = self.model.predict(X)
            scores = accuracy_score(y, preds)
            
            return 1 - scores

    # @timeout(900)
    def main():
        GBT = XGBTWrapper()

        facades: list[AbstractFacade] = []
        for intensifier_object in [Hyperband]:

            scenario = Scenario(
                GBT.configspace,
                walltime_limit=3600,
                output_directory=Path("smac_hyperband_output_budget_1hr_XGBT_337/" + dataset_names[i]),
                n_trials=10000,
                min_budget=100,
                max_budget=1000,
                n_workers=8,

            )

            initial_design = MFFacade.get_initial_design(scenario, n_configs=5)
            intensifier = intensifier_object(scenario, incumbent_selection="highest_budget")

            smac = MFFacade(
                scenario,
                GBT.fit,
                initial_design=initial_design,
                intensifier=intensifier,
                overwrite=True,
            )

            print("optimizing")
            # print(type(smac), "|", smac)
            incumbent = smac.optimize()
            best_params = incumbent.get_dictionary()
            params_dict_XGBT[dataset_names[i]] = best_params

            incumbent_cost = smac.runhistory.get_cost(incumbent)
            incumbent_run_id = incumbent.config_id

            print(f"Parameters: {best_params}")
            print(f"Cost: {incumbent_cost} | Config ID: {incumbent_run_id}")

            default_cost = smac.validate(GBT.configspace.get_default_configuration())
            # print(f"Default cost ({intensifier.__class__.__name__}): {default_cost}")
            incumbent_cost = smac.validate(incumbent)
            # print(f"Incumbent cost ({intensifier.__class__.__name__}): {incumbent_cost}")

            facades.append(smac)
        #     for arrt in dir(smac):
        #         if not arrt.startswith("_"):
        #             print(arrt, getattr(smac, arrt))

        # print("facades:", facades)



    if __name__ == "__main__":
        # with open('smac_results_2h.txt', "w") as f:
        #     pass
        # profiler = LineProfiler()
        # profiler.add_function(main)
        # profiler.enable()

        main()

        # profiler.disable()
        # profiler.print_stats()
    

[INFO][abstract_initial_design.py:82] Using `n_configs` and ignoring `n_configs_per_hyperparameter`.
[INFO][abstract_initial_design.py:147] Using 5 initial design configurations and 0 additional configurations.
optimizing
[INFO][successive_halving.py:164] Successive Halving uses budget type BUDGETS with eta 3, min budget 100, and max budget 1000.
[INFO][successive_halving.py:323] Number of configs in stage:
[INFO][successive_halving.py:325] --- Bracket 0: [9, 3, 1]
[INFO][successive_halving.py:325] --- Bracket 1: [5, 1]
[INFO][successive_halving.py:325] --- Bracket 2: [3]
[INFO][successive_halving.py:327] Budgets in stage:
[INFO][successive_halving.py:329] --- Bracket 0: [111.1111111111111, 333.3333333333333, 1000.0]
[INFO][successive_halving.py:329] --- Bracket 1: [333.3333333333333, 1000.0]
[INFO][successive_halving.py:329] --- Bracket 2: [1000.0]
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] F

2024-09-21 01:43:39,481 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://127.0.0.1:36645', name: 5, status: init, memory: 0, processing: 0>
2024-09-21 01:43:39,487 - distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:36645
2024-09-21 01:43:39,489 - distributed.core - INFO - Starting established connection to tcp://127.0.0.1:55862
2024-09-21 01:43:39,492 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://127.0.0.1:38669', name: 2, status: init, memory: 0, processing: 0>
2024-09-21 01:43:39,495 - distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:38669
2024-09-21 01:43:39,497 - distributed.core - INFO - Starting established connection to tcp://127.0.0.1:55842
2024-09-21 01:43:39,502 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://127.0.0.1:46335', name: 4, status: init, memory: 0, processing: 0>
2024-09-21 01:43:39,505 - distributed.scheduler - INFO - Starting worker compute

[INFO][abstract_initial_design.py:147] Using 5 initial design configurations and 0 additional configurations.
optimizing
[INFO][successive_halving.py:164] Successive Halving uses budget type BUDGETS with eta 3, min budget 100, and max budget 1000.
[INFO][successive_halving.py:323] Number of configs in stage:
[INFO][successive_halving.py:325] --- Bracket 0: [9, 3, 1]
[INFO][successive_halving.py:325] --- Bracket 1: [5, 1]
[INFO][successive_halving.py:325] --- Bracket 2: [3]
[INFO][successive_halving.py:327] Budgets in stage:
[INFO][successive_halving.py:329] --- Bracket 0: [111.1111111111111, 333.3333333333333, 1000.0]
[INFO][successive_halving.py:329] --- Bracket 1: [333.3333333333333, 1000.0]
[INFO][successive_halving.py:329] --- Bracket 2: [1000.0]
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] Finished 0 trials.
[INFO

2024-09-21 03:44:10,866 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/worker.py", line 1252, in heartbeat
    response = await retry_operation(
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/utils_comm.py", line 452, in retry_operation
    return await retry(
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/utils_comm.py", line 431, in retry
    return await coro()
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/py

[INFO][abstract_intensifier.py:594] Added config c7d68e and rejected config 3b9ad3 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][smbo.py:319] Finished 50 trials.
[INFO][smbo.py:319] Finished 50 trials.
[INFO][smbo.py:319] Finished 100 trials.
[INFO][smbo.py:319] Finished 100 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 450 trials.
[INFO][smbo.py:319] Finished 600 trials.
[INFO][smbo.py:319] Finished 1000 trials.
[INFO][smbo.py:319] Finished 1050 trials.
[INFO][smbo.py:319] Finished 1050 trials.
[INFO][smbo.py:319] Finished 1100 trials.
[INFO][smbo.py:319] Finished 1200 trials.
[INFO][smbo.py:319] Finished 1400 trials.
[INFO][smbo.py:319]

2024-09-21 06:44:17,567 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/worker.py", line 1252, in heartbeat
    response = await retry_operation(
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/utils_comm.py", line 452, in retry_operation
    return await retry(
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/utils_comm.py", line 431, in retry
    return await coro()
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/py

[INFO][abstract_intensifier.py:515] Added config dc624a as new incumbent because there are no incumbents yet.
[INFO][abstract_intensifier.py:594] Added config 160a93 and rejected config dc624a as incumbent because it is not better than the incumbents on 1 instances:
[INFO][smbo.py:319] Finished 50 trials.
[INFO][smbo.py:319] Finished 200 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 450 trials.
[INFO][smbo.py:319] Finished 550 trials.
[INFO][smbo.py:319] Finished 600 trials.
[INFO][smbo.py:319] Finished 650 trials.
[INFO][smbo.py:319] Finished 1000 trials.
[INFO][smbo.py:319] Finished 1000 trials.
[INFO][smbo.py:319] Finished 1000 trials.
[INFO][smbo.py:319] Finished 1000 

2024-09-21 17:04:42,356 - distributed.scheduler - INFO - Remove client Client-181a0878-7852-11ef-8927-246e9624e1b0
2024-09-21 17:04:42,358 - distributed.core - INFO - Received 'close-stream' from tcp://127.0.0.1:40116; closing.
2024-09-21 17:04:42,364 - distributed.scheduler - INFO - Remove client Client-181a0878-7852-11ef-8927-246e9624e1b0
2024-09-21 17:04:42,368 - distributed.scheduler - INFO - Close client connection: Client-181a0878-7852-11ef-8927-246e9624e1b0
2024-09-21 17:04:42,375 - distributed.scheduler - INFO - Retire worker addresses (0, 1, 2, 3, 4, 5, 6, 7)
2024-09-21 17:04:42,379 - distributed.nanny - INFO - Closing Nanny at 'tcp://127.0.0.1:34023'. Reason: nanny-close
2024-09-21 17:04:42,383 - distributed.nanny - INFO - Nanny asking worker to close. Reason: nanny-close
2024-09-21 17:04:42,384 - distributed.nanny - INFO - Closing Nanny at 'tcp://127.0.0.1:33845'. Reason: nanny-close
2024-09-21 17:04:42,387 - distributed.nanny - INFO - Nanny asking worker to close. Reason: n

In [41]:
for dataset, params in params_dict_XGBT.items():
    print(f"Dataset: {dataset}, Best Parameters: {params}")
    # print(f"Best Parameters: {params}")
with open("SmacResults/337/XGBT_params_1hr.json", 'w') as f:
    json.dump(params_dict_XGBT, f, indent=4)

Dataset: credit, Best Parameters: {'colsample_bylevel': 0.9701000734994785, 'colsample_bynode': 0.6545724822936216, 'colsample_bytree': 0.5958614671469253, 'gamma': 0.14035844350103893, 'learning_rate': 0.28687548834719934, 'max_depth': 8, 'min_child_weight': 2, 'n_estimators': 1494, 'subsample': 0.6912214484095188}
Dataset: electricity, Best Parameters: {'colsample_bylevel': 0.683119850018741, 'colsample_bynode': 0.8595439292367302, 'colsample_bytree': 0.9936804816043545, 'gamma': 0.17841636973138664, 'learning_rate': 0.2996472843928155, 'max_depth': 19, 'min_child_weight': 2, 'n_estimators': 1094, 'subsample': 0.684939858432469}
Dataset: covertype, Best Parameters: {'colsample_bylevel': 0.6841694527491273, 'colsample_bynode': 0.7521258791466592, 'colsample_bytree': 0.8542075266578653, 'gamma': 0.17841636973138664, 'learning_rate': 0.2936068843275964, 'max_depth': 14, 'min_child_weight': 3, 'n_estimators': 965, 'subsample': 0.679753950286893}
Dataset: pol, Best Parameters: {'colsample

# RF

In [ ]:
class RFWrapper(BaseEstimator):
    def __init__(self, n_estimators=100, max_depth=2, random_state=317, min_samples_split=2, min_samples_leaf=1, max_features=None, criterion="gini", max_samples = 0.5):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features   
        self.criterion = "gini"
        self.max_samples = max_samples
        self.model = RandomForestClassifier(n_estimators=self.n_estimators, max_depth=self.max_depth, random_state=self.random_state, min_samples_split=self.min_samples_split, min_samples_leaf=self.min_samples_leaf, criterion=self.criterion, max_features=self.max_features, max_samples=self.max_samples)

    @property
    def configspace(self) -> ConfigurationSpace:
        cs = ConfigurationSpace()
        n_estimators = Integer("n_estimators", (100, 1200), default=100)
        max_depth = Integer("max_depth", (2,21), default=2)
        min_samples_split = Integer("min_samples_split", (2, 20), default=2)
        min_samples_leaf = Integer("min_samples_leaf", (1, 20), default=1)
        criterion = Categorical("criterion", ["gini", "entropy", "log_loss"], default="gini")
        max_features = Categorical("max_features", ["sqrt", "log2", "None"], default="None")
        max_samples = Float("max_samples", (0.1, 0.99), log=True)
        cs.add_hyperparameters([n_estimators, max_depth, min_samples_split, min_samples_leaf, criterion, max_features, max_samples])
        return cs
    
    def fit(self, config: Configuration, seed: int = 0, budget: int = 250) -> float: 
        config = dict(config)  
        if config['max_features'] == 'None':
            config['max_features'] = None
        self.model.set_params(**config)
        X = train_x
        y = train_y
        self.model.fit(X, y)
        preds = self.model.predict(X)
        scores = accuracy_score(y, preds)

        return 1 - scores

In [ ]:
@timeout(900)
def main():
    RF = RFWrapper()

    facades: list[AbstractFacade] = []
    for intensifier_object in [Hyperband]:

        scenario = Scenario(
            RF.configspace,
            walltime_limit=600,
            output_directory=Path("smac_hyperband_output_budget_10mins_RF"),
            n_trials=10000,
            min_budget=100,
            max_budget=1000,
            n_workers=8,

        )

        initial_design = MFFacade.get_initial_design(scenario, n_configs=5)
        intensifier = intensifier_object(scenario, incumbent_selection="highest_budget")

        smac = MFFacade(
            scenario,
            RF.fit,
            initial_design=initial_design,
            intensifier=intensifier,
            overwrite=True,
        )

        print("optimiizing")
        print(type(smac), "|", smac)
        incumbent = smac.optimize()
        print("incumbent:", incumbent)
        default_cost = smac.validate(RF.configspace.get_default_configuration())
        print(f"Default cost ({intensifier.__class__.__name__}): {default_cost}")
        incumbent_cost = smac.validate(incumbent)
        print(f"Incumbent cost ({intensifier.__class__.__name__}): {incumbent_cost}")

        facades.append(smac)
        for arrt in dir(smac):
            if not arrt.startswith("_"):
                print(arrt, getattr(smac, arrt))

    print("facades:", facades)



if __name__ == "__main__":
    # with open('smac_results_2h.txt', "w") as f:
    #     pass
    profiler = LineProfiler()
    profiler.add_function(main)
    profiler.enable()

    main()

    profiler.disable()
    profiler.print_stats()

    

In [43]:
params_dict_RF = {}
for i in range(len(train_x_list)):
    train_x = train_x_list[i]
    train_y = train_y_list[i]
    class RFWrapper(BaseEstimator):
        def __init__(self, n_estimators=100, max_depth=2, random_state=317, min_samples_split=2, min_samples_leaf=1, max_features=None, criterion="gini", max_samples = 0.5):
            self.n_estimators = n_estimators
            self.max_depth = max_depth
            self.random_state = random_state
            self.min_samples_split = min_samples_split
            self.min_samples_leaf = min_samples_leaf
            self.max_features = max_features   
            self.criterion = "gini"
            self.max_samples = max_samples
            self.model = RandomForestClassifier(n_estimators=self.n_estimators, max_depth=self.max_depth, random_state=self.random_state, min_samples_split=self.min_samples_split, min_samples_leaf=self.min_samples_leaf, criterion=self.criterion, max_features=self.max_features, max_samples=self.max_samples)

        @property
        def configspace(self) -> ConfigurationSpace:
            cs = ConfigurationSpace()
            n_estimators = Integer("n_estimators", (100, 1500), default=100)
            max_depth = Integer("max_depth", (2,21), default=2)
            min_samples_split = Integer("min_samples_split", (2, 20), default=2)
            min_samples_leaf = Integer("min_samples_leaf", (1, 20), default=1)
            criterion = Categorical("criterion", ["gini", "entropy", "log_loss"], default="gini")
            max_features = Categorical("max_features", ["sqrt", "log2", "None"], default="None")
            max_samples = Float("max_samples", (0.1, 0.99), log=True)
            cs.add_hyperparameters([n_estimators, max_depth, min_samples_split, min_samples_leaf, criterion, max_features, max_samples])
            return cs
        
        def fit(self, config: Configuration, seed: int = 0, budget: int = 250) -> float: 
            config = dict(config)  
            if config['max_features'] == 'None':
                config['max_features'] = None
            self.model.set_params(**config)
            X = train_x
            y = train_y
            self.model.fit(X, y)
            preds = self.model.predict(X)
            scores = accuracy_score(y, preds)

            return 1 - scores

    # @timeout(90)
    def main():
        RF = RFWrapper()

        facades: list[AbstractFacade] = []
        for intensifier_object in [Hyperband]:

            scenario = Scenario(
                RF.configspace,
                walltime_limit=3600,
                output_directory=Path("smac_hyperband_output_budget_1hr_RF_337/" + dataset_names[i]),
                n_trials=10000,
                min_budget=100,
                max_budget=1000,
                n_workers=8,

            )

            initial_design = MFFacade.get_initial_design(scenario, n_configs=5)
            intensifier = intensifier_object(scenario, incumbent_selection="highest_budget")

            smac = MFFacade(
                scenario,
                RF.fit,
                initial_design=initial_design,
                intensifier=intensifier,
                overwrite=True,
            )

            print("optimiizing")
            # print(type(smac), "|", smac)
            incumbent = smac.optimize()
            best_params = incumbent.get_dictionary()
            params_dict_RF[dataset_names[i]] = best_params

            incumbent_cost = smac.runhistory.get_cost(incumbent)
            incumbent_run_id = incumbent.config_id

            print(f"Parameters: {best_params}")
            print(f"Cost: {incumbent_cost} | Config ID: {incumbent_run_id}")

            default_cost = smac.validate(RF.configspace.get_default_configuration())
            # print(f"Default cost ({intensifier.__class__.__name__}): {default_cost}")
            incumbent_cost = smac.validate(incumbent)
            # print(f"Incumbent cost ({intensifier.__class__.__name__}): {incumbent_cost}")

            facades.append(smac)
        #     for arrt in dir(smac):
        #         if not arrt.startswith("_"):
        #             print(arrt, getattr(smac, arrt))

        # print("facades:", facades)



    if __name__ == "__main__":
        # with open('smac_results_2h.txt', "w") as f:
        #     pass
        # profiler = LineProfiler()
        # profiler.add_function(main)
        # profiler.enable()

        main()

        # profiler.disable()
        # profiler.print_stats()

    
    

[INFO][abstract_initial_design.py:82] Using `n_configs` and ignoring `n_configs_per_hyperparameter`.
[INFO][abstract_initial_design.py:147] Using 5 initial design configurations and 0 additional configurations.
optimiizing
[INFO][successive_halving.py:164] Successive Halving uses budget type BUDGETS with eta 3, min budget 100, and max budget 1000.
[INFO][successive_halving.py:323] Number of configs in stage:
[INFO][successive_halving.py:325] --- Bracket 0: [9, 3, 1]
[INFO][successive_halving.py:325] --- Bracket 1: [5, 1]
[INFO][successive_halving.py:325] --- Bracket 2: [3]
[INFO][successive_halving.py:327] Budgets in stage:
[INFO][successive_halving.py:329] --- Bracket 0: [111.1111111111111, 333.3333333333333, 1000.0]
[INFO][successive_halving.py:329] --- Bracket 1: [333.3333333333333, 1000.0]
[INFO][successive_halving.py:329] --- Bracket 2: [1000.0]
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] 

2024-09-22 00:58:18,798 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/worker.py", line 1252, in heartbeat
    response = await retry_operation(
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/utils_comm.py", line 452, in retry_operation
    return await retry(
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/utils_comm.py", line 431, in retry
    return await coro()
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/py

[INFO][abstract_intensifier.py:594] Added config 0d93a9 and rejected config f64af0 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][smbo.py:319] Finished 50 trials.
[INFO][abstract_intensifier.py:594] Added config b773ba and rejected config 0d93a9 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:594] Added config eda38b and rejected config b773ba as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:594] Added config 99f2eb and rejected config eda38b as incumbent because it is not better than the incumbents on 1 instances:
[INFO][smbo.py:319] Finished 250 trials.
[INFO][smbo.py:319] Finished 300 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INF

2024-09-22 01:59:12,536 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/worker.py", line 1252, in heartbeat
    response = await retry_operation(
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/utils_comm.py", line 452, in retry_operation
    return await retry(
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/utils_comm.py", line 431, in retry
    return await coro()
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/py

[INFO][abstract_intensifier.py:594] Added config 4411a2 and rejected config 5b1b2f as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:594] Added config e3b1be and rejected config 4411a2 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][smbo.py:319] Finished 50 trials.
[INFO][abstract_intensifier.py:594] Added config 33245a and rejected config e3b1be as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:594] Added config 88365f and rejected config 33245a as incumbent because it is not better than the incumbents on 1 instances:
[INFO][smbo.py:319] Finished 100 trials.
[INFO][smbo.py:319] Finished 150 trials.
[INFO][smbo.py:319] Finished 200 trials.
[INFO][smbo.py:319] Finished 250 trials.
[INFO][smbo.py:319] Finished 300 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 400 trials.
[INFO][smbo.py:319] Finished 450 trials.
[INF

...

2024-09-22 11:24:31,371 - distributed.core - INFO - Event loop was unresponsive in Nanny for 3.23s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-09-22 11:24:31,372 - distributed.core - INFO - Event loop was unresponsive in Nanny for 3.23s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-09-22 11:24:31,507 - distributed.core - INFO - Event loop was unresponsive in Nanny for 3.28s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-09-22 11:24:31,510 - distributed.core - INFO - Event loop was unresponsive in Nanny for 3.28s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-09-22 11:24:31,513 - distributed.core - INFO - Event loop was u

[INFO][abstract_intensifier.py:515] Added config e04e5a as new incumbent because there are no incumbents yet.
[INFO][abstract_intensifier.py:594] Added config e3d7df and rejected config e04e5a as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:594] Added config ec275e and rejected config e3d7df as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:594] Added config 0a69c8 and rejected config ec275e as incumbent because it is not better than the incumbents on 1 instances:
[INFO][smbo.py:319] Finished 100 trials.
[INFO][smbo.py:319] Finished 100 trials.
[INFO][smbo.py:319] Finished 100 trials.
[INFO][smbo.py:319] Finished 150 trials.
[INFO][smbo.py:319] Finished 200 trials.
[INFO][smbo.py:319] Finished 300 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo.py:319] Finished 400 trials.
[INFO][smbo.py:319] Finished 450 trials.
[INFO][smbo.py:319] Finished 500 trials.
[INFO][sm

2024-09-22 13:30:34,756 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/worker.py", line 1252, in heartbeat
    response = await retry_operation(
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/utils_comm.py", line 452, in retry_operation
    return await retry(
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/distributed/utils_comm.py", line 431, in retry
    return await coro()
  File "/home/ziyan/miniconda3/envs/NeuroData/lib/py

[INFO][smbo.py:319] Finished 50 trials.
[INFO][abstract_intensifier.py:594] Added config d83666 and rejected config 32655b as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:594] Added config 3dc15a and rejected config d83666 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:594] Added config c8c7d6 and rejected config 3dc15a as incumbent because it is not better than the incumbents on 1 instances:
[INFO][smbo.py:319] Finished 150 trials.
[INFO][abstract_intensifier.py:594] Added config a32ff8 and rejected config c8c7d6 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:594] Added config ca9d08 and rejected config a32ff8 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][smbo.py:319] Finished 200 trials.
[INFO][smbo.py:319] Finished 300 trials.
[INFO][smbo.py:319] Finished 350 trials.
[INFO][smbo

2024-09-22 14:50:38,268 - distributed.scheduler - INFO - Remove client Client-f44b848d-7907-11ef-8927-246e9624e1b0
2024-09-22 14:50:38,270 - distributed.core - INFO - Received 'close-stream' from tcp://127.0.0.1:37522; closing.
2024-09-22 14:50:38,273 - distributed.scheduler - INFO - Remove client Client-f44b848d-7907-11ef-8927-246e9624e1b0
2024-09-22 14:50:38,275 - distributed.scheduler - INFO - Close client connection: Client-f44b848d-7907-11ef-8927-246e9624e1b0
2024-09-22 14:50:38,279 - distributed.scheduler - INFO - Retire worker addresses (0, 1, 2, 3, 4, 5, 6, 7)
2024-09-22 14:50:38,282 - distributed.nanny - INFO - Closing Nanny at 'tcp://127.0.0.1:39547'. Reason: nanny-close
2024-09-22 14:50:38,286 - distributed.nanny - INFO - Nanny asking worker to close. Reason: nanny-close
2024-09-22 14:50:38,287 - distributed.nanny - INFO - Closing Nanny at 'tcp://127.0.0.1:33317'. Reason: nanny-close
2024-09-22 14:50:38,290 - distributed.nanny - INFO - Nanny asking worker to close. Reason: n

In [23]:
# params_dict_RF_1hr = {}
name = "road-safety"
runhistory_path = "smac_hyperband_output_budget_1hr_RF_334/road-safety/80369a6b8358032cab31c23967a470da/0/runhistory.json"

with open(runhistory_path, "r") as file:
    runhistory_data_CNN = json.load(file)

print("Length of runhistory_data_CNN:", len(runhistory_data_CNN["data"]))

configs_costs_CNN = []
for entry in runhistory_data_CNN["data"]:
    # print(entry)
    config_id = entry[0]  
    cost = entry[4]  
    configs_costs_CNN.append((entry[0], entry[4]))
    configs_costs_CNN = list(set(configs_costs_CNN))

min_cost_config_CNN = sorted(configs_costs_CNN, key=lambda x: x[1])[0]
print("CNN min 1:")
config_id, cost = min_cost_config_CNN
params = runhistory_data_CNN["configs"][str(config_id)]
print(f"Config ID: {config_id}, Cost: {cost}, Parameters: {params}\n")
params_dict_RF_1hr[name] = params
file.close()
print(params_dict_RF_1hr)


Length of runhistory_data_CNN: 1077
CNN min 1:
Config ID: 521, Cost: 0.0, Parameters: {'criterion': 'entropy', 'max_depth': 21, 'max_features': 'None', 'max_samples': 0.9812660700637297, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 320}

{'albert': {'criterion': 'gini', 'max_depth': 17, 'max_features': 'None', 'max_samples': 0.9727926495978603, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 859}, 'compas-two-years': {'criterion': 'gini', 'max_depth': 18, 'max_features': 'log2', 'max_samples': 0.9723218124088577, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 224}, 'covertype': {'criterion': 'gini', 'max_depth': 19, 'max_features': 'None', 'max_samples': 0.9875527991780877, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 696}, 'default-of-credit-card-clients': {'criterion': 'gini', 'max_depth': 21, 'max_features': 'None', 'max_samples': 0.9858398960835011, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 703}

In [46]:
for dataset_name, params in params_dict_RF.items():
    print(f"Dataset: {dataset_name}, Best Parameters: {params}")
    # print(f"Best Parameters: {params}")
with open("SmacResults/337/RF_params_1hr.json", 'w') as f:
    json.dump(params_dict_RF, f, indent=4)

Dataset: credit, Best Parameters: {'criterion': 'entropy', 'max_depth': 20, 'max_features': 'sqrt', 'max_samples': 0.9691083523958117, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 252}
Dataset: electricity, Best Parameters: {'criterion': 'gini', 'max_depth': 20, 'max_features': 'log2', 'max_samples': 0.9647413210898548, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 108}
Dataset: covertype, Best Parameters: {'criterion': 'gini', 'max_depth': 21, 'max_features': 'sqrt', 'max_samples': 0.9847607906550341, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 302}
Dataset: pol, Best Parameters: {'criterion': 'entropy', 'max_depth': 21, 'max_features': 'sqrt', 'max_samples': 0.9706978963446904, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 317}
Dataset: house_16H, Best Parameters: {'criterion': 'log_loss', 'max_depth': 21, 'max_features': 'log2', 'max_samples': 0.984461385589925, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_e

# TabNet

In [ ]:
class TabWrapper(BaseEstimator):
    def __init__(self, n_d=64, n_a=64, n_steps=5, gamma=1.3, n_independent=2, n_shared=2, seed=317, optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=1e-2), scheduler_params={"step_size":50, "gamma":0.9}, scheduler_fn=torch.optim.lr_scheduler.StepLR, mask_type='entmax', verbose=0):
        self.n_d = n_d
        self.n_a = n_a
        self.n_steps = n_steps
        self.gamma = gamma
        self.n_independent = n_independent
        self.n_shared = n_shared
        self.seed = seed
        self.optimizer_fn = optimizer_fn
        self.optimizer_params = optimizer_params
        self.scheduler_params = scheduler_params
        self.scheduler_fn = scheduler_fn
        self.mask_type = mask_type
        self.model = TabNetClassifier(n_d=self.n_d, n_a=self.n_a, n_steps=self.n_steps, gamma=self.gamma, n_independent=self.n_independent, n_shared=self.n_shared, seed=self.seed, optimizer_fn=self.optimizer_fn, optimizer_params=self.optimizer_params, scheduler_params=self.scheduler_params, scheduler_fn=self.scheduler_fn, mask_type=self.mask_type)

    @property
    def configspace(self) -> ConfigurationSpace:
        cs = ConfigurationSpace()
        n_d = Integer("n_d", (4, 256), default=64)
        n_a = Integer("n_a", (4, 256), default=64)
        # n_steps = Integer("n_steps", (3, 10), default=5)
        # gamma = Float("gamma", (0.9, 2.0), default=1.3)
        # n_independent = Integer("n_independent", (1, 10), default=2)
        # n_shared = Integer("n_shared", (1, 10), default=2)
        # seed = Integer("seed", (0, 1000), default=317)
        # optimizer_fn = Categorical("optimizer_fn", [torch.optim.Adam, torch.optim.AdamW], default=torch.optim.Adam)
        # scheduler_fn = Categorical("scheduler_fn", [torch.optim.lr_scheduler.StepLR, torch.optim.lr_scheduler.MultiStepLR], default=torch.optim.lr_scheduler.StepLR)
        # mask_type = Categorical("mask_type", ['sparsemax', 'entmax'], default='entmax')
        cs.add_hyperparameters([n_d, n_a])
        # cs.add_hyperparameters([n_d, n_a, n_steps, gamma, n_independent, n_shared, seed, optimizer_fn, scheduler_fn, mask_type])
        return cs

    def fit(self, config: Configuration, seed: int = 0, budget: int = 250) -> float:
        config = dict(config)
        self.model.set_params(**config)
        X = train_x
        y = train_y
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        self.model.fit(X_train, y_train, eval_set=[(X_val, y_val)], patience=50)
        preds = self.model.predict(X_val)
        score = accuracy_score(y_val, preds)
        return 1 - score

In [ ]:
@timeout(900)
def main():
    Tab = TabWrapper()

    facades: list[AbstractFacade] = []
    for intensifier_object in [Hyperband]:

        scenario = Scenario(
            Tab.configspace,
            walltime_limit=600,
            output_directory=Path("smac_hyperband_output_budget_10mins_Tab"),
            n_trials=10000,
            min_budget=100,
            max_budget=1000,
            n_workers=8,

        )

        initial_design = MFFacade.get_initial_design(scenario, n_configs=5)
        intensifier = intensifier_object(scenario, incumbent_selection="highest_budget")

        smac = MFFacade(
            scenario,
            Tab.fit,
            initial_design=initial_design,
            intensifier=intensifier,
            overwrite=True,
        )

        print("optimiizing")
        print(type(smac), "|", smac)
        incumbent = smac.optimize()
        print("incumbent:", incumbent)
        default_cost = smac.validate(Tab.configspace.get_default_configuration())
        print(f"Default cost ({intensifier.__class__.__name__}): {default_cost}")
        incumbent_cost = smac.validate(incumbent)
        print(f"Incumbent cost ({intensifier.__class__.__name__}): {incumbent_cost}")

        facades.append(smac)
        for arrt in dir(smac):
            if not arrt.startswith("_"):
                print(arrt, getattr(smac, arrt))

    print("facades:", facades)

if __name__ == "__main__":
    # with open('smac_results_2h.txt', "w") as f:
    #     pass
    profiler = LineProfiler()
    profiler.add_function(main)
    profiler.enable()

    main()

    profiler.disable()
    profiler.print_stats()

In [ ]:
params_dict_Tab = {}
for i in range(len(train_x_list)):
    train_x = train_x_list[i]
    train_y = train_y_list[i]
    class TabWrapper(BaseEstimator):
        def __init__(self, n_d=8, n_a=8, n_steps=5, gamma=1.3, n_independent=2, n_shared=2, seed=317, optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=1e-2), mask_type='entmax', verbose=0):
            self.n_d = n_d
            self.n_a = n_a
            self.n_steps = n_steps
            self.gamma = gamma
            self.n_independent = n_independent
            self.n_shared = n_shared
            self.seed = seed
            self.optimizer_fn = optimizer_fn
            self.optimizer_params = optimizer_params
            self.mask_type = mask_type
            self.verbose = 0
            self.model = TabNetClassifier(n_d=self.n_d, n_a=self.n_a, n_steps=self.n_steps, gamma=self.gamma, n_independent=self.n_independent, n_shared=self.n_shared, seed=self.seed, optimizer_fn=self.optimizer_fn, optimizer_params=self.optimizer_params, mask_type=self.mask_type, verbose=self.verbose)

        @property
        def configspace(self) -> ConfigurationSpace:
            cs = ConfigurationSpace()
            n_d = Integer("n_d", (4, 128), default=8)
            n_a = Integer("n_a", (4, 128), default=8)
            n_steps = Integer("n_steps", (3, 5), default=5)
            gamma = Float("gamma", (1, 2.0), default=1.3)
            n_independent = Integer("n_independent", (1, 10), default=2)
            n_shared = Integer("n_shared", (1, 10), default=2)
            optimizer_fn = Categorical("optimizer_fn", [torch.optim.Adam, torch.optim.AdamW], default=torch.optim.Adam)
            # mask_type = Categorical("mask_type", ['sparsemax', 'entmax'], default='entmax')
            # seed = Integer("seed", (317), default=317)
            verbose = Categorical("verbose", [0], default=0)
            cs.add_hyperparameters([n_d, n_a, verbose])
            # cs.add_hyperparameters([n_d, n_a, n_steps, gamma, n_independent, n_shared, seed, optimizer_fn, scheduler_fn, mask_type])
            return cs

        def fit(self, config: Configuration, seed: int = 0, budget: int = 250) -> float:
            # config = dict(config)
            # self.model.set_params(**config)
            self.model = TabNetClassifier(n_d=config["n_d"], n_a=config["n_a"], verbose=self.verbose)
            X = train_x
            y = train_y
            X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
            self.model.fit(X_train, y_train, eval_set=[(X_val, y_val)], patience=10)
            preds = self.model.predict(X_val)
            score = accuracy_score(y_val, preds)
            return 1 - score    

    @timeout(2100)
    def main():
        start_time = time.time()
        print("Here 1")
        Tab = TabWrapper()

        facades: list[AbstractFacade] = []
        for intensifier_object in [Hyperband]:

            scenario = Scenario(
                Tab.configspace,
                walltime_limit=1800,
                output_directory=Path("smac_hyperband_output_budget_30mins_Tab/" + dataset_names[i]),
                n_trials=10000,
                min_budget=100,
                max_budget=1000,
                n_workers=8,

            )
            

            initial_design = MFFacade.get_initial_design(scenario, n_configs=5)
            intensifier = intensifier_object(scenario, incumbent_selection="highest_budget")

            smac = MFFacade(
                scenario,
                Tab.fit,
                initial_design=initial_design,
                intensifier=intensifier,
                overwrite=True,
            )

            print("optimizing")
            # print(type(smac), "|", smac)
            incumbent = smac.optimize()
            best_params = incumbent.get_dictionary()
            params_dict_Tab[dataset_names[i]] = best_params

            print("Here 3")
            incumbent_cost = smac.runhistory.get_cost(incumbent)
            incumbent_run_id = incumbent.config_id

            print(f"Parameters: {best_params}")
            print(f"Cost: {incumbent_cost} | Config ID: {incumbent_run_id}")

            # if time.time() - start_time > 60:
            #     break

            default_cost = smac.validate(Tab.configspace.get_default_configuration())
            # print(f"Default cost ({intensifier.__class__.__name__}): {default_cost}")
            incumbent_cost = smac.validate(incumbent)
            # print(f"Incumbent cost ({intensifier.__class__.__name__}): {incumbent_cost}")

            facades.append(smac)
        #     for arrt in dir(smac):
        #         if not arrt.startswith("_"):
        #             print(arrt, getattr(smac, arrt))

        # print("facades:", facades)



    if __name__ == "__main__":
        # with open('smac_results_2h.txt', "w") as f:
        #     pass
        # profiler = LineProfiler()
        # profiler.add_function(main)
        # profiler.enable()

        main()

        break

        # profiler.disable()


Early stopping occurred at epoch 22 with best_epoch = 12 and best_val_0_auc = 0.84879


/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [6]:
params_dict_Tab = {}

In [16]:

# for i in range(len(train_x_list)):
i = 15
train_x = train_x_list[i]
train_y = train_y_list[i]
print(dataset_names[i])
class TabWrapper(BaseEstimator):
    def __init__(self):
        self.model = TabNetClassifier(device_name='cuda' if torch.cuda.is_available() else 'cpu', verbose=0)
        self.max_epochs = 100
        self.batch_size = 1024
        self.optimizer_fn = torch.optim.Adam
        self.optimizer_params = dict(lr=1e-2)
        self.n_d = 8
        self.n_a = 8

    @property
    def configspace(self) -> ConfigurationSpace:
        cs = ConfigurationSpace()
        n_d = Integer("n_d", (4, 128), default=8)
        n_a = Integer("n_a", (4, 128), default=8)
        max_epochs = Integer("max_epochs", (50, 300), default=100)
        batch_size = Integer("batch_size", (64, 2048), default=1024)
        solver = Categorical("solver", ["sgd", "adam"])
        # optimizer_fn = Categorical("optimizer_fn", [torch.optim.Adam, torch.optim.SGD], default=torch.optim.Adam)
        lr = Float("lr", (0.001, 0.1), default=1e-2)
        cs.add_hyperparameters([n_d, n_a, max_epochs, batch_size, solver, lr])
        return cs

    def fit(self, config: Configuration, seed: int = 0, budget: int = 250) -> float:
        self.lr = config.get("lr")
        self.optimizer_params.update({"lr": self.lr})
        self.n_d = config.get("n_d")
        self.n_a = config.get("n_a")
        self.max_epochs = config.get("max_epochs")
        self.batch_size = config.get("batch_size")
        solver_name = config.get("solver")
        if solver_name == "adam":
            self.optimizer_fn = torch.optim.Adam
        else:
            self.optimizer_fn = torch.optim.SGD
        self.model = TabNetClassifier(n_d=self.n_d, n_a=self.n_a, optimizer_params=self.optimizer_params, optimizer_fn=self.optimizer_fn, verbose=0)
        X = train_x
        y = train_y
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=317)
        self.model.fit(X_train, y_train, eval_set=[(X_val, y_val)], max_epochs=self.max_epochs, patience=10, batch_size=self.batch_size)
        preds = self.model.predict(X_val)
        score = accuracy_score(y_val, preds)
        return 1 - score
        # return None


@timeout(4000)
def main():
    start_time = time.time()
    Tab = TabWrapper()

    facades: list[AbstractFacade] = []
    for intensifier_object in [Hyperband]:

        scenario = Scenario(
            Tab.configspace,
            walltime_limit=3600,
            output_directory=Path("smac_hyperband_output_budget_1hr_Tab_337/" + dataset_names[i]),
            n_trials=10000,
            min_budget=100,
            max_budget=1000,
            n_workers=8,

        )
        

        initial_design = MFFacade.get_initial_design(scenario, n_configs=5)
        intensifier = intensifier_object(scenario, incumbent_selection="highest_budget")

        smac = MFFacade(
            scenario,
            Tab.fit,
            initial_design=initial_design,
            intensifier=intensifier,
            overwrite=True,
        )

        print("optimizing")
        # print(type(smac), "|", smac)
        incumbent = smac.optimize()
        best_params = incumbent.get_dictionary()
        params_dict_Tab[dataset_names[i]] = best_params

        print("Here 3")
        incumbent_cost = smac.runhistory.get_cost(incumbent)
        incumbent_run_id = incumbent.config_id

        print(f"Parameters: {best_params}")
        print(f"Cost: {incumbent_cost} | Config ID: {incumbent_run_id}")

        # if time.time() - start_time > 60:
        #     break

        default_cost = smac.validate(Tab.configspace.get_default_configuration())
        # print(f"Default cost ({intensifier.__class__.__name__}): {default_cost}")
        incumbent_cost = smac.validate(incumbent)
        # print(f"Incumbent cost ({intensifier.__class__.__name__}): {incumbent_cost}")

        facades.append(smac)
    #     for arrt in dir(smac):
    #         if not arrt.startswith("_"):
    #             print(arrt, getattr(smac, arrt))

    # print("facades:", facades)



if __name__ == "__main__":
    # with open('smac_results_2h.txt', "w") as f:
    #     pass
    # profiler = LineProfiler()
    # profiler.add_function(main)
    # profiler.enable()

    main()
    # break

    # profiler.disable()

heloc
[INFO][abstract_initial_design.py:82] Using `n_configs` and ignoring `n_configs_per_hyperparameter`.
[INFO][abstract_initial_design.py:147] Using 5 initial design configurations and 0 additional configurations.
optimizing
[INFO][successive_halving.py:164] Successive Halving uses budget type BUDGETS with eta 3, min budget 100, and max budget 1000.
[INFO][successive_halving.py:323] Number of configs in stage:
[INFO][successive_halving.py:325] --- Bracket 0: [9, 3, 1]
[INFO][successive_halving.py:325] --- Bracket 1: [5, 1]
[INFO][successive_halving.py:325] --- Bracket 2: [3]
[INFO][successive_halving.py:327] Budgets in stage:
[INFO][successive_halving.py:329] --- Bracket 0: [111.1111111111111, 333.3333333333333, 1000.0]
[INFO][successive_halving.py:329] --- Bracket 1: [333.3333333333333, 1000.0]
[INFO][successive_halving.py:329] --- Bracket 2: [1000.0]
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:

/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version 


Early stopping occurred at epoch 10 with best_epoch = 0 and best_val_0_auc = 0.49855

Early stopping occurred at epoch 12 with best_epoch = 2 and best_val_0_auc = 0.58152


/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 10 with best_epoch = 0 and best_val_0_auc = 0.4455


...


Early stopping occurred at epoch 40 with best_epoch = 30 and best_val_0_auc = 0.77922


/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 59 with best_epoch = 49 and best_val_0_auc = 0.79928


/home/ziyan/miniconda3/envs/NeuroData/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Here 3
Parameters: {'batch_size': 1694, 'lr': 0.0807610762686905, 'max_epochs': 90, 'n_a': 5, 'n_d': 21, 'solver': 'adam'}
Cost: 0.255 | Config ID: 441

Early stopping occurred at epoch 20 with best_epoch = 10 and best_val_0_auc = 0.52452

Early stopping occurred at epoch 51 with best_epoch = 41 and best_val_0_auc = 0.78491


In [32]:
import json

name = "MagicTelescope"
runhistory_path = "smac_hyperband_output_budget_1hr_Tab_337/MagicTelescope/20d8c6b9231ce935bedd925f3db2c5e8/0/runhistory.json"

with open(runhistory_path, "r") as file:
    runhistory_data_CNN = json.load(file)

print("Length of runhistory_data_CNN:", len(runhistory_data_CNN["data"]))

configs_costs_CNN = []
for entry in runhistory_data_CNN["data"]:
    # print(entry)
    config_id = entry[0]  
    cost = entry[4]  
    configs_costs_CNN.append((entry[0], entry[4]))
    configs_costs_CNN = list(set(configs_costs_CNN))

    # print(f"Config ID: {config_id}, Cost: {cost}")
    # if config_id == 44:# or config_id == 21:
    #     params_CNN = runhistory_data_CNN["configs"][str(config_id)]
    #     # print(entry)
    #     print(f"CNN config ID: {config_id}, Cost: {cost}, Parameters: {params_CNN}\n")


# min_cost_config_CNN = []
# min_cost_config_CNN = sorted(configs_costs_CNN, key=lambda x: x[1])[:3]
# print("CNN min 3:")
# for config_id, cost in min_cost_config_CNN:
    
#     params = runhistory_data_CNN["configs"][str(config_id)]
#     print(f"Config ID: {config_id}, Cost: {cost}, Parameters: {params}\n")
min_cost_config_CNN = sorted(configs_costs_CNN, key=lambda x: x[1])[0]
print("CNN min 1:")
config_id, cost = min_cost_config_CNN
params = runhistory_data_CNN["configs"][str(config_id)]
print(f"Config ID: {config_id}, Cost: {cost}, Parameters: {params}\n")
params_dict_Tab[name] = params
file.close()
print(len(params_dict_Tab))

Length of runhistory_data_CNN: 58
CNN min 1:
Config ID: 33, Cost: 0.14149999999999996, Parameters: {'batch_size': 352, 'lr': 0.05526518620015621, 'max_epochs': 59, 'n_a': 65, 'n_d': 12, 'solver': 'adam'}

16


In [33]:
print(dataset_names)
# print(type(params_dict_Tab))
print(len(params_dict_Tab))

for dataset_name, params in params_dict_Tab.items():
    print(dataset_name, ":", params)

with open('SmacResults/337/Tab_params_1hr.json', 'w') as f:
    json.dump(params_dict_Tab, f, indent=4)

['credit', 'electricity', 'covertype', 'pol', 'house_16H', 'MagicTelescope', 'bank-marketing', 'MiniBooNE', 'Higgs', 'eye_movements', 'Diabetes130US', 'jannis', 'default-of-credit-card-clients', 'Bioresponse', 'california', 'heloc']
16
bank-marketing : {'batch_size': 185, 'lr': 0.004368846756807461, 'max_epochs': 114, 'n_a': 102, 'n_d': 123, 'solver': 'adam'}
Bioresponse : {'batch_size': 130, 'lr': 0.052352220817896866, 'max_epochs': 204, 'n_a': 74, 'n_d': 96, 'solver': 'adam'}
california : {'batch_size': 427, 'lr': 0.018175686711437454, 'max_epochs': 246, 'n_a': 83, 'n_d': 4, 'solver': 'adam'}
covertype : {'batch_size': 264, 'lr': 0.014660013598153034, 'max_epochs': 253, 'n_a': 63, 'n_d': 5, 'solver': 'adam'}
credit : {'batch_size': 1022, 'lr': 0.07590230482035659, 'max_epochs': 68, 'n_a': 28, 'n_d': 121, 'solver': 'adam'}
default-of-credit-card-clients : {'batch_size': 1707, 'lr': 0.08475030485508209, 'max_epochs': 196, 'n_a': 97, 'n_d': 125, 'solver': 'adam'}
Diabetes130US : {'batch

In [5]:
print(params_dict_Tab.keys())

NameError: name 'params_dict_Tab' is not defined